In [1]:
import numpy as np
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline


Using TensorFlow backend.


In [2]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

# load dataset
nps = pd.read_csv("/Users/01017387/Desktop/しごと/data/car用/test/sampled_million_for_feature_sele.csv", sep="\t",nrows=100)

#削除対象の変数を省く
drop_list = ["data_cat","hm_cd","brand_cd","fmc_cd","shashu_cd","keisai_days","shinkoyo_nohimbi_cd", "shinkoyo_nohimbi_cd",
         "good_madoguchi_flg", "del_cd", "Unnamed: 0", "Unnamed: 0.1","grade_cd", "chiho_x",  "mc_cd","grade_kukuri_cd_x",
         "body_type_cd", "city_name","shashu_x", "address", "ask_price_flg", "seiyaku_flg",
         "del_date", "y_other", "p_other", "y_cs", "p_cs", "wns_bukken_cd","predicted_price"]

dr2 = nps.filter(like='hojin_cd', axis=1).columns.tolist()
dr3 = nps.filter(like='madoguchi_cd', axis=1).columns.tolist()

nps = nps.drop(drop_list,axis=1)
nps = nps.drop(dr2,axis=1)
nps = nps.drop(dr3,axis=1).fillna(0)

# dataset = nps.values
# split into input (X) and output (Y) variables
X = nps.drop("y",axis=1).astype(float).values
Y = nps["y"].values


In [3]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)

In [4]:
# !python -V
#condaで3.5環境を作成する必要ありそう

In [5]:
#1. baseline model
def create_baseline():
    # create model
    model = Sequential()
    model.add(Dense(X.shape[1], input_dim=X.shape[1], init='normal', activation='relu'))
    model.add(Dense(1, init='normal', activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# evaluate model with standardized dataset
estimator = KerasClassifier(build_fn=create_baseline, nb_epoch=100, batch_size=5, verbose=0)
# kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
# results = cross_val_score(estimator, X, encoded_Y, cv=kfold)
estimator.fit(X, encoded_Y)
# print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

estimator.score(X, encoded_Y)
prob = estimator.predict_proba(X, encoded_Y)

TypeError: predict_proba() takes 2 positional arguments but 3 were given

In [ ]:
#2. evaluate baseline model with standardized dataset
np.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_baseline, nb_epoch=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Standardized: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))# evaluate baseline model with standardized dataset

np.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_baseline, nb_epoch=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Standardized: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

In [ ]:
#3. smaller model
def create_smaller():
    # create model
    model = Sequential()
    model.add(Dense(len(nps.columns)/2, input_dim=len(nps.columns), init='normal', activation='relu'))
    model.add(Dense(1, init='normal', activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

np.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_smaller, nb_epoch=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Smaller: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

In [ ]:
#4. larger model
def create_larger():
    # create model
    model = Sequential()
    model.add(Dense(len(nps.columns), input_dim=len(nps.columns), init='normal', activation='relu'))
    model.add(Dense(len(nps.columns)/2, init='normal', activation='relu'))
    model.add(Dense(1, init='normal', activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

np.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_larger, nb_epoch=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Larger: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))